In [ ]:
%load_ext autoreload
%autoreload 1

import datasets
from transformers import GPT2TokenizerFast
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import TrainingArguments, Trainer
from transformers import GPT2LMHeadModel
import os

if not os.path.isdir("trained_models"):
    os.mkdir("trained_models")

dataset = datasets.load_dataset(
    "ucberkeley-dlab/measuring-hate-speech",
    "binary"
)["train"]

# print(type(dataset[dataset["hate_speech_score"] > 0.5]["text"]))

# dataset = dataset[dataset["target_race"] == True]
# hate_speech_dataset = dataset[dataset["hate_speech_score"] > 0.5]["text"].tolist()
# supportive_dataset = dataset[dataset["hate_speech_score"] < -1]["text"].tolist()

# print(len(hate_speech_dataset))

# multiple seeds
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
model_id = "gpt2"


In [ ]:
# class HatespeechDataset(Dataset):
#     def __init__(self, encodings):
#         self.encodings = encodings

#     def __getitem__(self, index):
#         item = {key: torch.tensor(val[index].clone().detach()) for key, val in self.encodings.items()}
#         return item

#     def __len__(self):
#         return self.encodings.input_ids.shape[0]

tokenizer = GPT2TokenizerFast.from_pretrained(
    model_id,
    # bos_token="<|startoftext|>", 
    # eos_token="<|endoftext|>",
    # pad_token="<|pad|>"
)

# train_encoding = tokenizer(
    # dataset, 
    # padding=True, 
    # truncation=True, 
    # max_length=1024, 
    # return_tensors="pt"
# )


def tokenize_function(examples):
    return tokenizer(examples['text'])

# train_dataset = HatespeechDataset(train_encoding)
train_dataset = dataset.map(tokenize_function, batched=True, batch_size=len(dataset))

training_args = TrainingArguments(
    output_dir="trained_models",
    seed=42
)

In [ ]:
model = GPT2LMHeadModel.from_pretrained(model_id).to(device)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    # eval_dataset=small_eval_dataset,
    # compute_metrics=compute_metrics,
)

trainer.train()

In [1]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel, TrainingArguments, Trainer, default_data_collator
import datasets
import torch
from itertools import chain

device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"

tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)

tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.max_length = 1024
tokenizer.truncation = True
tokenizer.padding = "max_length"

dataset = datasets.load_dataset(
    "ucberkeley-dlab/measuring-hate-speech",
    "binary"
)["train"]

dataset = dataset.filter(lambda example: example["target_race"] == True)
hate_speech_dataset = dataset.filter(lambda example: example["hate_speech_score"] > 0.5)

# columns = hate_speech_dataset.column_names
# columns.remove("text")

# hate_speech_dataset = hate_speech_dataset.remove_columns(columns) 

hate_speech_dataset.to_json("hate_speech_dataset.json")

supportive_speech_dataset = dataset.filter(lambda example: example["hate_speech_score"] < -1)

supportive_speech_dataset.to_json("supportive_speech_dataset.json")

# def data_collator(features):
#     tokenized_features = tokenizer(features)
#     batch = {}
#     batch["input_ids"] = torch.stack([f[0] for f in features])
#     batch["attention_mask"] = torch.stack([f[1] for f in features])

#     return batch

# training_args = TrainingArguments(
#     output_dir="trained_models",
#     seed=42,
#     save_strategy="epoch",
#     logging_strategy="epoch"
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=hate_speech_dataset,
#     tokenizer=tokenizer,
#     data_collator=default_data_collator,
# )
# # Finetune the model
# trainer.train()

/home/andreib/miniconda3/envs/bias_measure/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset parquet (/home/andreib/.cache/huggingface/datasets/ucberkeley-dlab___parquet/ucberkeley-dlab--measuring-hate-speech-c32713cabe528196/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 1/1 [00:00<00:00, 21.78it/s]
Loading cached processed dataset at /home/andreib/.cache/huggingface/datasets/ucberkeley-dlab___parquet/ucberkeley-dlab--measuring-hate-speech-c32713cabe528196/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-f655188cf9fee53d.arrow
Loading cached processed dataset at /home/andreib/.cache/huggingface/datasets/ucberkeley-dlab___parquet/ucberkeley-dlab--measuring-hate-speech-c32713cabe528196/0.0.0/2a3b91fbd88a2c90d1dbbb32b

65694588